In [5]:
datadir = '/data/BIDS-HPC/private/projects/dmi2/data/all_gene_expression_files_in_target/links'
file_fpkm = 'fffee315-9aa3-44d2-8c89-78a2c1d107e7.FPKM.txt'
file_fpkm_uq = 'fffee315-9aa3-44d2-8c89-78a2c1d107e7.FPKM-UQ.txt'
file_counts = 'fffee315-9aa3-44d2-8c89-78a2c1d107e7.htseq_counts.txt'
gdc_tsv_file = '/data/BIDS-HPC/private/projects/dmi2/data/gencode.gene.info.v22.tsv'
gencode_gtf_file = '/data/BIDS-HPC/private/projects/dmi2/data/gencode.v22.annotation-just_genes.gtf'

In [18]:
import pandas as pd
import os
df_fpkm = pd.read_csv(os.path.join(datadir, file_fpkm), sep='\t', names=['id','fpkm'])
df_fpkm_uq = pd.read_csv(os.path.join(datadir, file_fpkm_uq), sep='\t', names=['id','fpkm-uq'])
df_counts = pd.read_csv(os.path.join(datadir, file_counts), sep='\t', skipfooter=5, names=['id','count'])
df_gdc = pd.read_csv(gdc_tsv_file, sep='\t')
df_gencode = pd.read_csv(gencode_gtf_file, sep='\t', skiprows=5, header=None)

In [3]:
# This line was edited but was used to show that the ID columns of all three dataframes are exactly the same
(df_fpkm_uq['id']==df_counts['id'])

0        True
1        True
2        True
3        True
4        True
         ... 
60478    True
60479    True
60480    True
60481    True
60482    True
Name: id, Length: 60483, dtype: bool

In [27]:
tol = 1e-6
((df_fpkm['fpkm']/df_fpkm['fpkm'].sum() - df_fpkm_uq['fpkm-uq']/df_fpkm_uq['fpkm-uq'].sum()).abs() < tol).sum() # This line shows that the FPKM and FPKM-UQ files have the same values up to a fixed normalization
((df_counts['count']/df_counts['count'].sum() - df_fpkm_uq['fpkm-uq']/df_fpkm_uq['fpkm-uq'].sum()).abs() < tol).sum() # this line shows that this is not true of the counts

43200

In [30]:
#set(df_fpkm['id']) - set(df_gdc['gene_id'])
set(df_fpkm['id']) == set(df_gdc['gene_id'])

True

In [68]:
import numpy as np
arr = df_fpkm['fpkm'].to_numpy()
quantiles = np.quantile(arr, [0,0.25,0.5,0.75,1])
#arr[quantiles[0]<arr and arr<quantiles[3]]
print(arr[(quantiles[3]<arr) & (arr<quantiles[4])].sum())
1/(df_fpkm['fpkm'].max()/df_fpkm_uq['fpkm-uq'].max())

331163.3667


26613.247395329272

In [45]:
arr.max()/2

3865.25345

In [29]:
pd.qcut(df_fpkm['fpkm'], 4)

ValueError: Bin edges must be unique: array([0.0000000e+00, 0.0000000e+00, 4.7900000e-02, 8.8105000e-01,
       7.7305069e+03]).
You can drop duplicate edges by setting the 'duplicates' kwarg

In [9]:
df_counts

,id,count
0,ENSG00000000003.13,11205
1,ENSG00000000005.5,334
2,ENSG00000000419.11,4022
3,ENSG00000000457.12,539
4,ENSG00000000460.15,898
...,...,...
60478,ENSGR0000275287.3,0
60479,ENSGR0000276543.3,0
60480,ENSGR0000277120.3,0
60481,ENSGR0000280767.1,0


In [7]:
df_gdc

,gene_id,gene_name,seqname,start,end,strand,gene_type,gene_status,havana_gene,full_length,exon_length,exon_num
0,ENSG00000223972.5,DDX11L1,chr1,11869,14409,+,transcribed_unprocessed_pseudogene,KNOWN,OTTHUMG00000000961.2,2541,1735,9
1,ENSG00000238009.5,RP11-34P13.7,chr1,89295,133723,-,lincRNA,NOVEL,OTTHUMG00000001096.2,44429,3726,17
2,ENSG00000230415.1,RP5-902P8.10,chr1,1275223,1280420,+,lincRNA,NOVEL,OTTHUMG00000002234.2,5198,513,5
3,ENSG00000236335.1,RP4-591L5.1,chr1,30409560,30411638,-,lincRNA,NOVEL,OTTHUMG00000003682.1,2079,507,3
4,ENSG00000213842.2,SUGT1P2,chr3,32752910,32753901,+,processed_pseudogene,KNOWN,OTTHUMG00000155904.1,992,992,1
...,...,...,...,...,...,...,...,...,...,...,...,...
60478,ENSG00000226955.1,AC104306.4,chr3,32592972,32593312,-,processed_pseudogene,KNOWN,OTTHUMG00000155901.1,341,341,1
60479,ENSG00000231552.1,IGBP1P3,chr3,32620903,32621902,-,processed_pseudogene,KNOWN,OTTHUMG00000155898.1,1000,1000,1
60480,ENSG00000237676.1,RPL30P4,chr3,32635222,32635554,-,processed_pseudogene,KNOWN,OTTHUMG00000155900.1,333,333,1
60481,ENSG00000182973.17,CNOT10,chr3,32685145,32773875,+,protein_coding,KNOWN,OTTHUMG00000130748.5,88731,4846,145


In [19]:
df_gencode

,0,1,2,3,4,5,6,7,8
0,chr1,HAVANA,gene,11869,14409,.,+,.,"gene_id ""ENSG00000223972.5""; gene_type ""transc..."
1,chr1,HAVANA,gene,14404,29570,.,-,.,"gene_id ""ENSG00000227232.5""; gene_type ""unproc..."
2,chr1,ENSEMBL,gene,17369,17436,.,-,.,"gene_id ""ENSG00000278267.1""; gene_type ""miRNA""..."
3,chr1,HAVANA,gene,29554,31109,.,+,.,"gene_id ""ENSG00000243485.3""; gene_type ""lincRN..."
4,chr1,ENSEMBL,gene,30366,30503,.,+,.,"gene_id ""ENSG00000274890.1""; gene_type ""miRNA""..."
...,...,...,...,...,...,...,...,...,...
60478,chrM,ENSEMBL,gene,14149,14673,.,-,.,"gene_id ""ENSG00000198695.2""; gene_type ""protei..."
60479,chrM,ENSEMBL,gene,14674,14742,.,-,.,"gene_id ""ENSG00000210194.1""; gene_type ""Mt_tRN..."
60480,chrM,ENSEMBL,gene,14747,15887,.,+,.,"gene_id ""ENSG00000198727.2""; gene_type ""protei..."
60481,chrM,ENSEMBL,gene,15888,15953,.,+,.,"gene_id ""ENSG00000210195.2""; gene_type ""Mt_tRN..."
